# sBERT

In [ ]:
from transformers import AutoModel, AutoTokenizer
import torch.nn as nn
import torch
from sources.models import sBERTRegressor, sBERTRegressorNew, RoBERTaRegressor

device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
model_name = "snunlp/KR-SBERT-V40K-klueNLI-augSTS" 
is_freeze = False  

model = sBERTRegressor(model_name, is_freeze).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

trained_model = sBERTRegressor(model_name, is_freeze).to(device)
model_path = "checkpoints/sBERT_2024-07-03 17:15:29.pth"
trained_model.load_state_dict(torch.load(model_path))

In [ ]:
example = "으아아아"
max_length = 128
encoded_text = tokenizer(example, return_tensors="pt",max_length=max_length,padding="max_length",truncation=True)
iids = encoded_text["input_ids"].to(device)
atm = encoded_text["attention_mask"].to(device)

In [ ]:
print("학습되지 않은 모델:",round(float(model(iids, atm)), 3), "점")
print("학습된 모델:",round(float(trained_model(iids, atm)), 3), "점")

In [ ]:
device = "cpu"

model_name = "snunlp/KR-SBERT-V40K-klueNLI-augSTS" 
is_freeze = True  

model = sBERTRegressorNew(model_name, is_freeze, version=2).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

trained_model = sBERTRegressorNew(model_name, is_freeze, version=2).to(device)
model_path = "checkpoints/sBERTNewV2_2024-07-04_13-36-30.pth"
trained_model.load_state_dict(torch.load(model_path))

In [ ]:
example = "정말요???/????????????????????????????????????????????????/????????"
max_length = 128
encoded_text = tokenizer(example, return_tensors="pt",max_length=max_length,padding="max_length",truncation=True)
iids = encoded_text["input_ids"].to(device)
atm = encoded_text["attention_mask"].to(device)

In [ ]:
print("학습되지 않은 모델:",round(float(model(iids, atm)), 3), "점")
print("학습된 모델:",round(float(trained_model(iids, atm)), 3), "점")

# RoBERTa

In [13]:
from transformers import AutoModel, AutoTokenizer
import torch.nn as nn
import torch
from sources.models import sBERTRegressor, sBERTRegressorNew, RoBERTaRegressor

device = "cpu"
is_freeze = True
max_length = 128
model_name = "klue/roberta-large" 
model_path = "checkpoints/RoBERTa-Large_2024-07-04_17-03-51.pth"  

model = RoBERTaRegressor(model_name, is_freeze).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

trained_model = RoBERTaRegressor(model_name, is_freeze).to(device)
trained_model.load_state_dict(torch.load(model_path))

Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**PRETRAINED MODEL FREEZE**


Some weights of RobertaModel were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


**PRETRAINED MODEL FREEZE**


<All keys matched successfully>

In [20]:
from huggingface_hub import hf_hub_download
import pandas as pd
from dotenv import load_dotenv
import os

load_dotenv()
api_key = os.getenv("HF_API_KEY")
repo_id = "SaeSSak/Conversation"
file_path = hf_hub_download(repo_id, "TestData.parquet", repo_type="dataset", use_auth_token=api_key)

# 파일 로드
df = pd.read_parquet(file_path)
example = list(df["음성인식결과"])

encoded_text = tokenizer(example, return_tensors="pt",max_length=max_length,padding="max_length",truncation=True)
iids = encoded_text["input_ids"].to(device)
atm = encoded_text["attention_mask"].to(device)

In [27]:
import torch

# 모델 예측
with torch.no_grad():
    predictions = trained_model(iids, atm)

# 그래디언트 추적 분리 및 NumPy 배열 변환
predictions_numpy = predictions.detach().numpy()

# 데이터프레임에 예측 결과 추가
df["나이점수"] = df["나이"] * 10
df["예측점수"] = predictions_numpy

df["오차"] = (df["나이점수"] - df["예측점수"]).abs()

# 각 나이별로 편차 계산
age_groups = df.groupby("나이")["오차"].mean().reset_index()
overall_deviation = df["오차"].mean()

In [36]:
for ag in age_groups.values:
    print("나이:", ag[0], "  오차:", round(ag[1], 2))
print("MAE:",overall_deviation)

나이: 3.0   오차: 25.99
나이: 4.0   오차: 22.94
나이: 5.0   오차: 16.83
나이: 6.0   오차: 8.08
나이: 7.0   오차: 12.01
나이: 8.0   오차: 5.8
나이: 9.0   오차: 10.25
나이: 10.0   오차: 18.97
MAE: 15.107299566268921


In [28]:
name = model_path.split("/")[-1].split(".")[0]
df.to_csv(f"./result/{name}.csv", index = False)

# 모델 실험

In [ ]:
device = "cpu"

In [ ]:
example = "안녕하세요."
max_length = 128
encoded_text = tokenizer(example, return_tensors="pt",max_length=max_length,padding="max_length",truncation=True)
iids = encoded_text["input_ids"].to(device)
atm = encoded_text["attention_mask"].to(device)

In [ ]:
model_name = "snunlp/KR-SBERT-V40K-klueNLI-augSTS" 
is_freeze = True 
sbert = AutoModel.from_pretrained(model_name).to(device)

In [ ]:
outputs = sbert(input_ids=iids, attention_mask=atm)

In [ ]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output.last_hidden_state  # First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [ ]:
iids

In [ ]:
token_embeddings = outputs.last_hidden_state

In [ ]:
token_embeddings = outputs.last_hidden_state
print(token_embeddings.shape)
input_mask_expanded = atm.unsqueeze(-1).expand(token_embeddings.size()).float()
print(input_mask_expanded.shape)
sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
print(sum_embeddings.shape)
sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
print(sum_embeddings.shape)
(sum_embeddings / sum_mask).shape